Imports

In [2]:
import os
import glob
import random
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from typing import List, Optional
from dataclasses import dataclass
from plotly.graph_objs import Figure
from itertools import combinations_with_replacement

Constants

In [3]:
PMC_CONVENTIONAL = 4362.47
PMC_OPERATIVE_69 = 2024.481
PMC_OPERATIVE_138 = 2513.486

PQ_QUANTITY = 49

YEARS = [2026, 2028, 2030, 2032, 2034, 2036]
PENETRATION_LEVELS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

HEAVY_LOAD = 1642.69
CHARGING_STATION_POWER = [114, 147, 180, 213]

POSSIBLE_LOAD_COMBINATIONS = list(
    np.unique(
        [sum(combination) for combination in list(combinations_with_replacement(CHARGING_STATION_POWER, 3))]
        )
    )

In [4]:
# sorted_combinations = \
#     sorted([combination for combination in list(
#         combinations_with_replacement(CHARGING_STATION_POWER, 3)
#         )], key= lambda x: sum(x))

# for combination in sorted_combinations:
#     print(f'{combination} = {sum(combination)}')

Classes and functions

In [5]:
@dataclass
class Bar:
    '''
    A dataclass that retains relevant information about system's bars.

    This dataclass stores information about system bars, including the file name, name, voltage class, and a DataFrame.

    Attributes:
        _file_name (str): The file name associated with the bar data.

    Properties:
        file_name (str): The file name associated with the bar data.
        name (str): The name extracted from the file name (excluding extension).
        voltage_class (str): The voltage class extracted from the file name.
        df (pd.DataFrame): A DataFrame containing the bar data.
    '''

    _file_name: str

    @property
    def file_name(self) -> str:
        '''str: The file name associated with the bar data.'''
        return self._file_name
    
    @property
    def id(self) -> str:
        '''str: The id extracted from the file name.'''
        return self._file_name[4:9]

    @property
    def name(self) -> str:
        '''str: The name extracted from the file name (excluding extension).'''
        return self._file_name[10:-4].strip()

    @property
    def voltage_class(self) -> str:
        '''str: The voltage class extracted from the file name.'''
        return self.name[-3:]

    @property
    def df(self) -> pd.DataFrame:
        '''pd.DataFrame: A DataFrame containing the bar data.'''
        return self.read_csv(self._file_name)
    
    @property
    def power_threshold(self) -> float:
        '''
        Get the power threshold for operational voltage based on a filtered DataFrame.

        Returns:
            float: Power threshold for operational voltage.
        '''
        filtered_df = self.df.loc[self.df[1].between(0.95, 0.96)] 
        
        if not filtered_df.empty:
            voltage_threshold_index = filtered_df[1].idxmin()
            power_threshold = filtered_df.at[voltage_threshold_index, 0]
        else:
            power_threshold = PMC_CONVENTIONAL

        return power_threshold
    
    def read_csv(self, _file_name: str) -> pd.DataFrame:
        '''
        Read bar data from a CSV file and return it as a DataFrame.

        Args:
            _file_name (str): The file name of the CSV file.

        Returns:
            pd.DataFrame: A DataFrame containing the bar data.
        '''
        df = pd.read_csv(_file_name, delimiter= ';', header= None)
        return df

    def __repr__(self) -> str:
        '''str: A string representation of the Bar instance.'''
        return '{} {} -- Voltage class: {} -- Power Threshold for Operational Voltage: {:.2f}' \
                    .format(self.id, self.name.strip(), self.voltage_class, self.power_threshold)
    
def get_random_sum(loads: List[int],
                   iterations: int
                   ) -> float:
    '''
    Calculate the sum of a random selection of load values in megawatts (MW).

    This function randomly selects a specified number of load values from the provided list and calculates their sum in megawatts (MW).

    Args:
        loads (List[int]): A list of load values in megawatts (MW).
        iterations (int): The number of iterations to repeat the random selection and summing process.

    Returns:
        float: The average sum of the randomly selected load values in megawatts (MW) across the specified number of iterations.
    '''
    
    total_sum = 0
    for _ in range(iterations):
        total_sum += random.choice(seq= loads)

    return total_sum / 1000

def increment_power(load: float,
                    increment_amount: int,
                    increment_percentage: float
                    ) -> List[float]:
    '''
    Increment a load value by a specified number and percentage multiple times.

    This function takes a starting load value and increments it by a specified number and percentage for a given number of times.

    Args:
        load (float): The initial load value.
        increment_amount (int): The number of times to increment the load.
        increment_percentage (float): The percentage by which to increment the load in each step.

    Returns:
        list: A list containing the incremented load values, including the initial load.
    '''
    incremented_values = [load]

    for _ in range(increment_amount):
        load *= (1 + increment_percentage)
        incremented_values.append(load)

    return incremented_values

def make_matrices(max_power: float,
                  load_value: List[int],
                  penetration_level: int,
                  increment: bool = False
                  ) -> tuple[List[List], List[List]]:
    '''
    Generate matrices of power and margin of stability based on max power, load, charging station power, and increment settings.

    This function calculates power and margin of stability matrices for a given load value and power value reference, considering various charging station power scenarios.

    Args
        max_power (float): Reference power value.
        load_value (float): The base load value.
        charging_station_power (list): A list of charging station power values.
        increment (bool, optional): Whether to increment load with charging station power scenarios (default is False).

    Returns:
        tuple: A tuple containing two lists of lists:
            - A power matrix: List of lists representing power values for different scenarios.
            - A margin of stability matrix: List of lists representing margin of stability values for different scenarios.
    '''
    pow_matrix = []
    vsm_matrix = []

    for i in range(PQ_QUANTITY):
        pow_row = []
        vsm_row = []

        for _ in range(200):
            charging_site_power = (i + 1) * get_random_sum(loads= POSSIBLE_LOAD_COMBINATIONS,
                                                           iterations= penetration_level)
            
            power_value = (load_value + charging_site_power) if increment else load_value

            vsm_value = 100.0 * ((max_power - power_value) / max_power)

            pow_row.append(power_value)
            vsm_row.append(vsm_value)

        pow_matrix.append(pow_row)
        vsm_matrix.append(vsm_row)
    
    return pow_matrix, vsm_matrix

def filter_by_voltage(bars: List[Bar],
                      voltage_class: Optional[str] = None
                      ) -> List[Bar]:
    '''
    Filter a list of Bar instances by their voltage_class attribute.

    Args:
        bars (List[Bar]): List of Bar instances to filter.
        voltage_class (str): The voltage class to filter by. Defaults to None.

    Returns:
        List[Bar]: List of filtered Bar instances.
    '''

    if voltage_class is None:
        return bars
    
    filtered_bars = [bar for bar in bars if bar.voltage_class == voltage_class]
    
    return filtered_bars

def matrix_to_list(target_matrix: List[List[float]]) -> List[float]:
    '''
    Turns a matrix into a list.

    Args:
        target_matrix (List[List[float]]): Matrix to be turned into a list.

    Returns:
        List[float]: List of elements of target_matrix.
    '''

    matrix_as_list = [element for row in target_matrix for element in row]

    return matrix_as_list

def save_as_csv(target_data, file_name, folder_path):
    '''
    Save a list of data to a CSV file.

    Args:
        target_data (list): The data to be saved.
        file_name (str): The name of the CSV file.
        folder_path (str): The path to the folder where the file will be saved.

    Returns:
        None
    '''
    file_path = os.path.join(folder_path, file_name)

    pd.DataFrame(target_data).to_csv(file_path, index= False, header= None)

def avg(input_list: List) -> float:
    average_value = sum(input_list) / len(input_list) 
    
    return average_value 

def conventional_heatmap(target_matrix: dict) -> Figure:
    '''
    Create a heatmap using Plotly Express.

    Parameters:
        target_matrix (dict): A nested dictionary containing matrix data.
        title (str): A string to be used as title.

    Returns:
        A Figure object.
    '''
    min_values_matrix = {}
    avg_values_matrix = {}
    max_values_matrix = {}

    for year in YEARS:
        min_values_matrix[year] = {}
        avg_values_matrix[year] = {}
        max_values_matrix[year] = {}

        for penetration_level in PENETRATION_LEVELS:
            current_matrix = target_matrix[year][penetration_level]
            
            min_value = min(min(current_matrix))
            avg_value = avg(matrix_to_list(current_matrix))
            max_value = max(max(current_matrix))

            if min_value < 0:
                min_value = 0

            if avg_value < 0:
                avg_value = 0

            if max_value < 0:
                max_value = 0

            min_values_matrix[year][penetration_level] = min_value
            avg_values_matrix[year][penetration_level] = avg_value 
            max_values_matrix[year][penetration_level] = max_value 

    avg_df = pd.DataFrame([(year, penetration_level, avg_value) 
                    for year, penetration_levels_dict in avg_values_matrix.items()
                    for penetration_level, avg_value in penetration_levels_dict.items()],
                    columns=['Years', 'Penetration Levels', 'Avg Values'])

    fig = px.imshow(avg_df.pivot(index= 'Penetration Levels',
                                 columns= 'Years',
                                 values= 'Avg Values'),
                    x= YEARS,
                    y= PENETRATION_LEVELS[::-1],
                    labels= dict(color= 'Valores <br> Médios'),
                    title= None,
                    color_continuous_scale= 'Portland',
                    aspect= 'auto')
    
    fig.update_layout(xaxis_title= 'Anos',
                      yaxis_title= 'Quantidade de Eletropostos')

    fig.update_layout(yaxis=dict(tickvals= list(range(0, 11)),
                                 ticktext= list(range(10, -1, -1))))
    fig.update_layout(xaxis=dict(tickvals= YEARS,
                                 ticktext= YEARS))

    for year in YEARS:
        for idx, penetration_level in enumerate(PENETRATION_LEVELS[::-1]):
            min_value = min_values_matrix[year][penetration_level]
            max_value = max_values_matrix[year][penetration_level]
            fig.add_annotation(x=year, y=idx, text='{:.2f}<br>{:.2f}'.format(max_value, min_value),
                            showarrow=False, font=dict(color='white'), align='center')

    fig.update_layout(height= 600,
                      width= 900)
    fig.update_layout(title_x= 0.5)

    return fig
    
def bar_group_heatmap(target_directory: str) -> Figure:
    '''
    Generate a heatmap from CSV files in the specified directory.

    Parameters:
    - target_directory (str): The path to the directory containing CSV files.
    - title (str): The title of the heatmap.

    Returns:
    - Figure: Plotly figure object representing the heatmap.
    '''

    files_dict = {year: {penetration_level: [] for penetration_level in PENETRATION_LEVELS} for year in YEARS}

    min_values_dict = {year: {penetration_level: 100 for penetration_level in PENETRATION_LEVELS} for year in YEARS}
    avg_values_dict = {year: {penetration_level: 0 for penetration_level in PENETRATION_LEVELS} for year in YEARS}
    max_values_dict = {year: {penetration_level: 0 for penetration_level in PENETRATION_LEVELS} for year in YEARS}

    for filename in os.listdir(target_directory):
        if filename.endswith('.csv'):
            _, _, year_from_file, _, _, penetration_level = filename.split('_')

            year_from_file = int(year_from_file)
            penetration_level = int(penetration_level.split('.')[0])

            files_dict[year_from_file][penetration_level].append(os.path.join(target_directory, filename))

    for year in YEARS:
        for penetration_level in PENETRATION_LEVELS:

            avg_values_list = []

            for file in files_dict[year][penetration_level]:
                data = pd.read_csv(file, header= None)

                min_value = max(0, data.min().min())
                avg_value = data.mean().mean()
                max_value = data.max().max()
                
                avg_values_list.append(avg_value) 

                if min_value < min_values_dict[year][penetration_level]:
                    min_values_dict[year][penetration_level] = min_value

                avg_values_dict[year][penetration_level] = avg(avg_values_list)

                if max_value > max_values_dict[year][penetration_level]:
                    max_values_dict[year][penetration_level] = max_value

    avg_values_df = pd.DataFrame(avg_values_dict).sort_index(ascending= True)

    fig = px.imshow(avg_values_df,
                    labels= dict(x= 'Anos',
                                y= 'Quantidade de Eletropostos',
                                color= 'Valores <br> Médios'),
                    x=avg_values_df.columns,
                    y=avg_values_df.index[::-1],
                    color_continuous_scale='Portland',
                    aspect= 'auto')

    fig.update_layout(yaxis= dict(tickvals= list(range(0, len(PENETRATION_LEVELS))),
                                ticktext= list(range(len(PENETRATION_LEVELS) - 1, -1, -1))))
    fig.update_layout(xaxis= dict(tickvals= YEARS,
                                ticktext= YEARS))

    for year in YEARS:
        for idx, penetration_level in enumerate(PENETRATION_LEVELS[::-1]):
            min_annotation = min_values_dict[year][penetration_level]
            max_annotation = max_values_dict[year][penetration_level]
            fig.add_annotation(
                x=year,
                y=idx,
                text= '{:.2f}<br>{:.2f}'.format(max_annotation, min_annotation),
                showarrow= False,
                font= dict(color='white'), align='center')
            
    fig.update_layout(height= 600,
                      width= 900)
    fig.update_layout(title_x= 0.5)

    return fig

def conventioal_housing_capacity_heatmap(target_directory: str) -> Figure:
    '''
    Generate a heatmap from CSV files in the specified directory.

    Parameters:
    - target_directory (str): The path to the directory containing CSV files.
    - title (str): The title of the heatmap.

    Returns:
    - Figure: Plotly figure object representing the heatmap.
    '''
    
    min_values_dict = {}
    avg_values_dict = {}
    max_values_dict = {}

    for desired_year in YEARS:
        pattern = f'*_{desired_year}_*.csv'
        files = glob.glob(os.path.join(target_directory, pattern))

        for file_path in files:
            housing_capacity = pd.read_csv(file_path, header= None)
            housing_capacity_as_list = housing_capacity.values.flatten().tolist()

            _, _, year_from_filename, _, iteration = os.path.basename(file_path).split('_')

            year_from_filename = int(year_from_filename)
            iteration = int(iteration.split('.')[0])

            if year_from_filename not in min_values_dict:
                min_values_dict[year_from_filename] = {}
                avg_values_dict[year_from_filename] = {}
                max_values_dict[year_from_filename] = {}

            min_values_dict[year_from_filename][iteration] = min(housing_capacity_as_list)
            avg_values_dict[year_from_filename][iteration] = int(round((avg(housing_capacity_as_list))))
            max_values_dict[year_from_filename][iteration] = max(housing_capacity_as_list)                

            for year, values in min_values_dict.items():
                sorted_iterations = sorted(values, key= values.get, reverse= True)

                min_values_dict[year] = {iteration_idx: values[iteration_idx] for iteration_idx in sorted_iterations}
                
            for year, values in max_values_dict.items():
                sorted_iterations = sorted(values, key= values.get, reverse= True)

                max_values_dict[year] = {iteration_idx: values[iteration_idx] for iteration_idx in sorted_iterations}

    avg_df = pd.DataFrame(avg_values_dict).sort_index(ascending= True)

    fig = px.imshow(avg_df,
                    labels= dict(x= "Anos", 
                                y= "Quantidade de Barras Alocadas", 
                                color= "Valores <br> Médios"),
                    x= avg_df.columns,
                    y= avg_df.index[::-1],
                    color_continuous_scale= "Portland",
                    aspect= 'auto')

    fig.update_layout(yaxis= dict(tickvals= list(range(0, PQ_QUANTITY + 1)),
                                ticktext=list(range(PQ_QUANTITY + 1, -1, -1))))
    fig.update_layout(xaxis= dict(tickvals=YEARS,
                                ticktext=YEARS))

    for year in YEARS:
        for iteration in sorted_iterations[::-1]:
            min_value = min_values_dict[year][iteration]
            max_value = max_values_dict[year][iteration]

            fig.add_annotation(x= year,
                            y= -iteration + PQ_QUANTITY + 1,
                            text= '{}<br>{}'.format(max_value, min_value),
                            showarrow= False,
                            font= dict(color='white'),
                            align= 'center')         

    fig.update_layout(height= 2400,
                    width= 900)

    fig.update_layout(title_x= 0.5)

    return fig

def housing_capacity_heatmap(target_directory: str) -> Figure:
    '''
    Generate a heatmap from CSV files in the specified directory.

    Parameters:
    - target_directory (str): The path to the directory containing CSV files.
    - title (str): The title of the heatmap.

    Returns:
    - Figure: Plotly figure object representing the heatmap.
    '''
    
    min_values_dict = {}
    avg_values_dict = {}
    max_values_dict = {}

    for desired_year in YEARS:
        pattern = f'*_{desired_year}_*.csv'
        files = glob.glob(os.path.join(target_directory, pattern))

        for file_path in files:
            housing_capacity = pd.read_csv(file_path, header= None)
            housing_capacity_as_list = housing_capacity.values.flatten().tolist()

            _, _, year_from_filename, _, iteration = os.path.basename(file_path).split('_')

            year_from_filename = int(year_from_filename)
            iteration = int(iteration.split('.')[0])

            if year_from_filename not in min_values_dict:
                min_values_dict[year_from_filename] = {}
                avg_values_dict[year_from_filename] = {}
                max_values_dict[year_from_filename] = {}

            min_values_dict[year_from_filename][iteration] = min(housing_capacity_as_list)
            avg_values_dict[year_from_filename][iteration] = int(round((avg(housing_capacity_as_list))))
            max_values_dict[year_from_filename][iteration] = max(housing_capacity_as_list)

            if year_from_filename in [2032, 2034, 2036]:
                min_values_dict[year_from_filename][iteration] = 0
                avg_values_dict[year_from_filename][iteration] = 0
                max_values_dict[year_from_filename][iteration] = 0 
                

            for year, values in min_values_dict.items():
                sorted_iterations = sorted(values, key= values.get, reverse= True)

                min_values_dict[year] = {iteration_idx: values[iteration_idx] for iteration_idx in sorted_iterations}
                
            for year, values in max_values_dict.items():
                sorted_iterations = sorted(values, key= values.get, reverse= True)

                max_values_dict[year] = {iteration_idx: values[iteration_idx] for iteration_idx in sorted_iterations}

    avg_df = pd.DataFrame(avg_values_dict).sort_index(ascending= True)

    fig = px.imshow(avg_df,
                    labels= dict(x= "Anos", 
                                y= "Quantidade de Barras Alocadas", 
                                color= "Valores <br> Médios"),
                    x= avg_df.columns,
                    y= avg_df.index[::-1],
                    color_continuous_scale= "Portland",
                    aspect= 'auto')

    fig.update_layout(yaxis= dict(tickvals= list(range(0, PQ_QUANTITY + 1)),
                                ticktext=list(range(PQ_QUANTITY + 1, -1, -1))))
    fig.update_layout(xaxis= dict(tickvals=YEARS,
                                ticktext=YEARS))

    for year in YEARS:
        for iteration in sorted_iterations[::-1]:
            min_value = min_values_dict[year][iteration]
            max_value = max_values_dict[year][iteration]

            if year in [2032, 2034, 2036]:
                min_value = 0
                max_value = 0

            fig.add_annotation(x= year,
                            y= -iteration + PQ_QUANTITY + 1,
                            text= '{}<br>{}'.format(max_value, min_value),
                            showarrow= False,
                            font= dict(color='white'),
                            align= 'center')         

    fig.update_layout(height= 2400,
                    width= 900)

    fig.update_layout(title_x= 0.5)

    return fig


Bar objects

In [6]:
bars = [Bar(file_path) for file_path in glob.glob(os.path.join(r'PQs', '*.csv'))]

Load defintions

In [7]:
# Load yearly growth of 4.1%
incremented_load = increment_power(HEAVY_LOAD, 5, 2 * 0.041)

In [8]:
# import numpy as np

# print(possible_values := [sum(combination) for combination in list(combinations_with_replacement(CHARGING_STATION_POWER, 3))])

# values = []

# for _ in range(2000):
#     value = get_random_sum(CHARGING_STATION_POWER, get_amount= 3, iterations= 1)

#     values.append(value)

# print(np.unique(values))
# print(np.unique(possible_values) / 1000)

Conventional VSM

In [8]:
# Filtering of bars in the target voltage groups.
bars_69kv = sorted(filter_by_voltage(bars= bars, voltage_class= '069'), key= lambda bar: bar.power_threshold)
bars_138kv = sorted(filter_by_voltage(bars= bars, voltage_class= '138'), key= lambda bar: bar.power_threshold)

selected_bars = sorted(bars_69kv + bars_138kv, key= lambda bar: (bar.name, bar.power_threshold))

In [10]:
# import csv

# # Assuming 'selected_bars' is a list of objects with 'name', 'voltage_class', and 'power_threshold' attributes

# # Specify the CSV file name
# csv_file = 'selected_bars.csv'

# # Write the data to the CSV file
# with open(csv_file, 'w', newline='') as file:
#     writer = csv.writer(file)

#     # Write the header
#     writer.writerow(['Name', 'Voltage Class', 'Power Threshold'])

#     # Write the data
#     for bar in selected_bars:
#         writer.writerow([bar.name, bar.voltage_class, bar.power_threshold])

# print(f'Data has been written to {csv_file}')


In [9]:
pow_matrix = {}
vsm_matrix = {}

for year in YEARS:

    load_data = incremented_load[YEARS.index(year)]

    pow_matrix[year] = {}
    vsm_matrix[year] = {}
    
    for penetration_level in PENETRATION_LEVELS:
        pow_matrix[year][penetration_level], \
        vsm_matrix[year][penetration_level] = make_matrices(max_power= PMC_CONVENTIONAL,
                                                            load_value= load_data,
                                                            penetration_level= penetration_level,
                                                            increment= True
                                                            )

        save_as_csv(target_data= vsm_matrix[year][penetration_level],
                    file_name= f'vsm_matrix_{year}_pen_level_{penetration_level}.csv',
                    folder_path = r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Conventional_VSMs')
        
        save_as_csv(target_data= pow_matrix[year][penetration_level],
                    file_name= f'pow_matrix_{year}_pen_level_{penetration_level}.csv',
                    folder_path = r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Power_for_conventional_VSMs')
        

In [10]:
pow_138_matrix = {}
vsm_138_matrix = {}

for year in YEARS:

    load_data = incremented_load[YEARS.index(year)]

    pow_138_matrix[year] = {}
    vsm_138_matrix[year] = {}
    
    for penetration_level in PENETRATION_LEVELS:
        pow_138_matrix[year][penetration_level], \
        vsm_138_matrix[year][penetration_level] = make_matrices(max_power= bars_138kv[0].power_threshold,
                                                                load_value= load_data,
                                                                penetration_level= penetration_level,
                                                                increment= True
                                                            )

        # save_as_csv(target_data= vsm_matrix[year][penetration_level],
        #             file_name= f'vsm_matrix_{year}_pen_level_{penetration_level}.csv',
        #             folder_path = r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Conventional_VSMs')
        
        # save_as_csv(target_data= pow_matrix[year][penetration_level],
        #             file_name= f'pow_matrix_{year}_pen_level_{penetration_level}.csv',
        #             folder_path = r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Power_for_conventional_VSMs')
        

In [11]:
pow_69_matrix = {}
vsm_69_matrix = {}

for year in YEARS:

    load_data = incremented_load[YEARS.index(year)]

    pow_69_matrix[year] = {}
    vsm_69_matrix[year] = {}
    
    for penetration_level in PENETRATION_LEVELS:
        pow_69_matrix[year][penetration_level], \
        vsm_69_matrix[year][penetration_level] = make_matrices(max_power= bars_69kv[0].power_threshold,
                                                               load_value= load_data,
                                                               penetration_level= penetration_level,
                                                               increment= True
                                                            )

        # save_as_csv(target_data= vsm_matrix[year][penetration_level],
        #             file_name= f'vsm_matrix_{year}_pen_level_{penetration_level}.csv',
        #             folder_path = r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Conventional_VSMs')
        
        # save_as_csv(target_data= pow_matrix[year][penetration_level],
        #             file_name= f'pow_matrix_{year}_pen_level_{penetration_level}.csv',
        #             folder_path = r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Power_for_conventional_VSMs')
        

In [26]:
# pow_matrix_69_reference = {}
# vsm_matrix_69_reference = {}

# for year in YEARS:

#     load_data = incremented_loads[YEARS.index(year)]

#     pow_matrix_69_reference[year] = {}
#     vsm_matrix_69_reference[year] = {}
    
#     for penetration_level in PENETRATION_LEVELS:
#         pow_matrix_69_reference[year][penetration_level], \
#         vsm_matrix_69_reference[year][penetration_level], = make_matrices(max_power= PMC_OPERATIVE_69,
#                                                                   load_value= load_data,
#                                                                   charging_station_power= charging_station_power,
#                                                                   penetration_level= penetration_level,
#                                                                   increment= True)

#         save_as_csv(target_data= vsm_matrix_69_reference[year][penetration_level],
#                     file_name= f'vsm_matrix_{year}_pen_level_{penetration_level}_69_op.csv',
#                     folder_path = r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\69_operative_VSMs')

In [27]:
# pow_matrix_138_reference = {}
# vsm_matrix_138_reference = {}

# for year in YEARS:

#     load_data = incremented_loads[YEARS.index(year)]

#     pow_matrix_138_reference[year] = {}
#     vsm_matrix_138_reference[year] = {}
    
#     for penetration_level in PENETRATION_LEVELS:
#         pow_matrix_138_reference[year][penetration_level], \
#         vsm_matrix_138_reference[year][penetration_level], = make_matrices(max_power= PMC_OPERATIVE_138,
#                                                                   load_value= load_data,
#                                                                   charging_station_power= charging_station_power,
#                                                                   penetration_level= penetration_level,
#                                                                   increment= True)

#         save_as_csv(target_data= vsm_matrix_138_reference[year][penetration_level],
#                     file_name= f'vsm_matrix_{year}_pen_level_{penetration_level}_138_op.csv',
#                     folder_path = r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\138_operative_VSMs')

Plots

In [12]:
fig = go.Figure()

voltage_to_be_filtered = None

filtered_bars = filter_by_voltage(bars, voltage_to_be_filtered)

for bar in filtered_bars:
    data = bar.df.T
    
    fig.add_trace(
        go.Scatter(x= data.values[0],
                   y= data.values[1],
                   mode= 'lines',
                   name= f'{bar.name}',
                   showlegend= False)
    )

fig.update_layout(
    title = '',
    xaxis_title= 'Potência (MW)',
    yaxis_title= 'Tensão Elétrica (pu)'
)

fig.update_xaxes(range= [1956.02, 4400])

fig.update_layout(height= 600,
                  width= 900)

fig.show()


In [29]:
# fig = go.Figure()

# vsm_per_year = []
# vsm_per_year_69 = []
# vsm_per_year_138 = []

# for year in YEARS:
#     vsm_per_year.append(vsm_matrix[year][0][0][0])
#     vsm_per_year_69.append(vsm_matrix_69_reference[year][0][0][0])
#     vsm_per_year_138.append(vsm_matrix_138_reference[year][0][0][0])

# fig.add_trace(
#     go.Scatter(x= YEARS,
#                y= vsm_per_year,
#                name= 'Conventional VSM')
# )

# fig.add_trace(
#     go.Scatter(x= YEARS,
#                y= vsm_per_year_69,
#                name= '69kV operative VSM')
# )

# fig.add_trace(
#     go.Scatter(x= YEARS,
#                y= vsm_per_year_138,
#                name= '138kV operative VSM')
# )

# fig.update_layout(
#     xaxis_title= 'Years',
#     yaxis_title= 'Voltage margin (%)',
#     title= 'Voltage Stability Margin Evolution Over the Years',
#     title_x= 0.5
# )

# fig.show()

In [13]:
fig = conventional_heatmap(target_matrix= vsm_matrix)

fig.show()

# fig2 = make_heatmap(target_matrix= vsm_matrix_69_reference,
#                     title= '69kV Operative Point Maintenance Voltage Stability Margin <br> Evolution with Penetration Levels Over the Years')

# fig2.show()                                     

# fig3 = make_heatmap(target_matri x= vsm_matrix_138_reference,
#                     title= '138kV Operative Point Maintenance Voltage Stability Margin <br> Evolution with Penetration Levels Over the Years')

# fig3.show()                                     

In [14]:
fig1 = conventional_heatmap(target_matrix= vsm_138_matrix)

fig1.show()                                     

In [15]:
fig1 = conventional_heatmap(target_matrix= vsm_69_matrix)

fig1.show()                                     

In [33]:
# # DATA COLLECTION DONE!!!

# for year in YEARS:
#     for penetration_level in PENETRATION_LEVELS:
#         for bar in bars_138kv:
#             vsm_values = pd.DataFrame(pow_matrix[year][penetration_level]) \
#                             .apply(lambda x: 100.0 * ((bar.power_threshold - x) / bar.power_threshold),
#                                    axis= 1)

#             save_as_csv(target_data= vsm_values,
#                         file_name= f'{bar.name}_year_{year}_pen_level_{penetration_level}.csv'.strip(),
#                         folder_path= rf'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Operative_VSMs_by_bar_138')


In [34]:
# # DATA COLLECTION DONE!!!

# for year in YEARS:
#     for penetration_level in PENETRATION_LEVELS:
#         for bar in bars_69kv:     
#             vsm_values = pd.DataFrame(pow_matrix[year][penetration_level]) \
#                            .apply(lambda x: 100.0 * ((bar.power_threshold - x) / bar.power_threshold),
#                                   axis= 1)

#             save_as_csv(target_data= vsm_values,
#                         file_name= f'{bar.name}_year_{year}_pen_level_{penetration_level}.csv'.strip(),
#                         folder_path= rf'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Operative_VSMs_by_bar_069')


In [16]:
fig = bar_group_heatmap(target_directory= r'Operative_VSMs_by_bar_138')

fig.show()

In [19]:
fig = bar_group_heatmap(target_directory= r'Operative_VSMs_by_bar_069')

fig.show()

Housing Capacity

In [ ]:
# # DATA COLLECTION DONE!!!

# for year in YEARS:
#     for penetration_level in PENETRATION_LEVELS:
#         avaible_power = pd.DataFrame(pow_matrix[year][penetration_level]) \
#                             .apply(lambda x: PMC_CONVENTIONAL - x)
    
#         save_as_csv(target_data= avaible_power,
#                     file_name= f'avaible_power_year_{year}_pen_level_{penetration_level}.csv',
#                     folder_path= r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Conventional_avaible_power')


In [ ]:
# fig = housing_capacity_heatmap(target_directory= r'Conventional_avaible_power',
#                                title= 'Housing Capacity Evolution <br> with Penetration Levels Over the Years'
#                                )

# fig.show()

In [ ]:
# power_threshold_of_selected_bars = [bar.power_threshold for bar in sorted_selected_bars]

# # Dictionaries for storage.
# avaible_power_dict = {}
# housing_capacity_dict = {}

# # Outer loop that iterates over the length of selected_bars list.
# for year in YEARS:
    
#     avaible_power_dict[year] = {}
#     housing_capacity_dict[year] = {}
    
#     # Inner loop that iterates over the years.
#     for index, bar in enumerate(selected_bars):

#         avaible_power_dict[year][bar.name] = {}
#         housing_capacity_dict[year][bar.name] = {}
        
#         avaible_power_list = []
#         amount_cs_list = []

#         # Innermost loop that repeats the avaible power calculation repeated times.
#         for _ in range(1000):

#             # The power threshold for normal operation limits is sampled from power_threshold_of_selected_bars list and then
#             # year_load is subtracted from it. This gives an random amount of power avaible for the charging stations to be
#             # stored in. 
#             sampled_power_threshold = random.sample(random.sample(population= power_threshold_of_selected_bars, k= index+1), k= 1)[0]
#             avaible_power = sampled_power_threshold - incremented_loads[YEARS.index(year)]

#             # The power of the charging stations package is calculated selection three values from CHARGING_STATION_POWER.
#             charging_stations = get_random_sum(loads= possible_load_combinations, iterations= 1)
            
#             # Having these values is possible to determinate the amount of charging stations that can be stored in the avaible power.
#             amount_cs = avaible_power // charging_stations
            
#             # All the calculated amounts are stored in a list.
#             avaible_power_list.append(avaible_power)
#             amount_cs_list.append(int(amount_cs))

#         # The list in stored in the previously created dictionary, grouped by year.
#         housing_capacity_dict[year][bar.name] = amount_cs_list
#         avaible_power_dict[year][bar.name] = avaible_power_list

#         save_as_csv(target_data= housing_capacity_dict[year][bar.name],
#                     file_name= f'housing_capacity_{year}_ref_{bar.name}.csv',
#                     folder_path = r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Housing_capacity')
        
#         save_as_csv(target_data= avaible_power_dict[year][bar.name],
#                     file_name= f'avaible_power_{year}_ref_{bar.name}.csv',
#                     folder_path = r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Avaible_power')

In [ ]:
# min_housing_capacity_per_year = [min(min(bar_data) for bar_data in year_data.values()) for year_data in housing_capacity_dict.values()]

# print(min_housing_capacity_per_year)

In [ ]:
# # Works for the selection of only one bar

# housing_capacity_dict = {}

# for year in YEARS:

#     housing_capacity_dict[year] = {}
#     housing_capacity_list = []

#     for _ in range(500):
#         target_bar = random.sample(sorted_selected_bars, k=1)[0]

#         avaible_power = target_bar.power_threshold - incremented_loads[YEARS.index(year)]

#         stations_amount = 0

#         while avaible_power >= 0:
#             # By subtracting the station power from the avaible power, it is included in the system.
#             avaible_power -= get_random_sum(loads= possible_load_combinations, iterations= 1)

#             # While avaible power is non-negative, a station can be stored 
#             stations_amount += 1

#             if avaible_power >= 0:
#                 non_negative_power = avaible_power
#                 non_negative_staion_amount = stations_amount
        
#         housing_capacity_list.append(non_negative_staion_amount)

#         housing_capacity_dict[year] = housing_capacity_list

# for year in YEARS:
#     print(year, len(housing_capacity_dict[year]), sorted(housing_capacity_dict[year]))

In [ ]:
# # Works for the selection of i bars

# housing_capacity_dict = {}

# for year in YEARS:
    
#     housing_capacity_dict[year] = {}
    
#     # Iterates over the amount of bars in the precomputed list.
#     for i in range(1, len(selected_bars) + 1):

#         housing_capacity_dict[year][i] = {}
#         housing_capacity_list = []

#         for _ in range(500):
            
#             # Samples i powers from the precomputed list
#             sampled_bars = random.sample(selected_bars, k= i)
#             avaible_power = []

#             # The sampled bars' power threshold are used to calculated an avaible power list
#             for bar in sampled_bars:
#                 avaible_power.append(PMC_CONVENTIONAL - incremented_load[YEARS.index(year)])
            
#             avaible_power_after_insertion = avaible_power.copy()

#             loop_iteration = 0

#             # Checks if any of the powers is equal or less than zero                                                                                                                             
#             while all(power >= 0 for power in avaible_power_after_insertion):
#                 station_power = get_random_sum(POSSIBLE_LOAD_COMBINATIONS, 1)
                
#                 # By subtracting the station power from the avaible power, it is included in the system load.
#                 avaible_power_after_insertion = [power - station_power for power in avaible_power_after_insertion]

#                 loop_iteration += 1

#             housing_capacity_list.append(last_possible_station_amount := loop_iteration - 1)

#         housing_capacity_dict[year][i] = housing_capacity_list

#         save_as_csv(target_data= housing_capacity_dict[year][i],
#                     file_name= f'housing_capacity_{year}_iteration_{i}.csv',
#                     folder_path = r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Housing_capacity_conventional')

In [ ]:
# # DATA COLLECTION DONE!!!

# # Works for the selection of i bars

# housing_capacity_dict = {}

# for year in YEARS:
    
#     housing_capacity_dict[year] = {}
    
#     # Iterates over the amount of bars in the precomputed list.
#     for i in range(1, len(selected_bars) + 1):

#         housing_capacity_dict[year][i] = {}
#         housing_capacity_list = []

#         for _ in range(500):
            
#             # Samples i powers from the precomputed list
#             sampled_bars = random.sample(selected_bars, k= i)
#             avaible_power = []

#             # The sampled bars' power threshold are used to calculated an avaible power list
#             for bar in sampled_bars:
#                 avaible_power.append(bar.power_threshold - incremented_load[YEARS.index(year)])
            
#             avaible_power_after_insertion = avaible_power.copy()

#             loop_iteration = 0

#             # Checks if any of the powers is equal or less than zero                                                                                                                             
#             while all(power >= 0 for power in avaible_power_after_insertion):
#                 station_power = get_random_sum(POSSIBLE_LOAD_COMBINATIONS, 1)
                
#                 # By subtracting the station power from the avaible power, it is included in the system load.
#                 avaible_power_after_insertion = [power - station_power for power in avaible_power_after_insertion]

#                 loop_iteration += 1

#             housing_capacity_list.append(last_possible_station_amount := loop_iteration - 1)

#         housing_capacity_dict[year][i] = housing_capacity_list

#         save_as_csv(target_data= housing_capacity_dict[year][i],
#                     file_name= f'housing_capacity_{year}_iteration_{i}.csv',
#                     folder_path = r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Housing_capacity_by_bar_threshold')

In [ ]:
# target_bar = random.sample(power_threshold_of_selected_bars, k= 4)

# avaible_power = [(lambda power: power - incremented_loads[0])(power) for power in target_bar]

# avaible_power_after_insertion = avaible_power

# loop_iteration = 0

# while all(power >= 0 for power in avaible_power_after_insertion):
#     loop_iteration += 1
    
#     station_power = get_random_sum(possible_load_combinations, 1)
#     avaible_power_after_insertion = [power - station_power for power in avaible_power_after_insertion]

#     last_possible_station_amount = loop_iteration - 1

#     print(loop_iteration, station_power, avaible_power_after_insertion)

# print(last_possible_station_amount)

In [42]:
fig = conventioal_housing_capacity_heatmap(target_directory= r'Housing_capacity_conventional')

fig.update_layout(height= 1600,
                  width= 900)

fig.show()

In [43]:
fig = housing_capacity_heatmap(target_directory= r'Housing_capacity_by_bar_threshold')

fig.update_layout(height= 1600,
                  width= 900)

fig.show()

In [ ]:
# target_directory= r'Operative_VSMs_by_bar_138'

# title= 'Operative Voltage Stability Margin Evolution <br> with Penetration Levels Over the Years for 138kV Bar Group'

# min_values_dict = {}
# avg_values_dict = {}
# max_values_dict = {}

# for desired_year in YEARS:
#     pattern = f'*_{desired_year}_*.csv'
#     files = glob.glob(os.path.join(target_directory, pattern))

#     for file_path in files:
#         df = pd.read_csv(file_path, header= None)

#         _, _, year_from_filename, _, _, pen_level_with_extension = os.path.basename(file_path).split('_')

#         year_from_filename = int(year_from_filename)
#         pen_level = int(pen_level_with_extension.split('.')[0])

#         if year_from_filename not in min_values_dict:
#             min_values_dict[year_from_filename] = {}
#             avg_values_dict[year_from_filename] = {}
#             max_values_dict[year_from_filename] = {}

#         min_values_dict[year_from_filename][pen_level] = df.min().min()
#         avg_values_dict[year_from_filename][pen_level] = df.mean().mean()
#         max_values_dict[year_from_filename][pen_level] = df.max().max()

#         for year, values in min_values_dict.items():
#             sorted_pen_levels = sorted(values,
#                                         key= values.get,
#                                         reverse= True)
#             min_values_dict[year] = {pen_level: values[pen_level] for pen_level in sorted_pen_levels}
        
#         for year, values in max_values_dict.items():
#             sorted_pen_levels = sorted(values,
#                                         key= values.get,
#                                         reverse= True)
#             max_values_dict[year] = {pen_level: values[pen_level] for pen_level in sorted_pen_levels}

# avg_df = pd.DataFrame(avg_values_dict).sort_index(ascending= True)

# fig = px.imshow(avg_df,
#                 labels= dict(x= "Pen Level",
#                              y= "Year",
#                              color= "Minimum Value"),
#                 x= avg_df.columns,
#                 y= avg_df.index[::-1],
#                 color_continuous_scale= "Portland",
#                 aspect= 'auto')

# fig.update_layout(title= title,
#                     xaxis_title= 'Anos',
#                     yaxis_title= 'Quantidade de Eletropostos')

# fig.update_layout(yaxis= dict(tickvals= list(range(0, len(PENETRATION_LEVELS))),
#                                 ticktext= list(range(len(PENETRATION_LEVELS), -1, -1))))
# fig.update_layout(xaxis= dict(tickvals= YEARS,
#                                 ticktext= YEARS))

# for year in YEARS:
#     for idx, penetration_level in enumerate(PENETRATION_LEVELS[::-1]):
#         min_value = min_values_dict[year][penetration_level]
#         max_value = max_values_dict[year][penetration_level]
#         fig.add_annotation(
#             x=year,
#             y=idx,
#             text= '{:.2f}<br>{:.2f}'.format(max_value, min_value),
#             showarrow= False,
#             font= dict(color='white'), align='center')
        
# fig.update_layout(height= 600,
#                   width= 900)

# fig.update_layout(title_x= 0.5)

# fig.show()


In [22]:
min_values_matrix = {}
avg_values_matrix = {}
max_values_matrix = {}

for year in YEARS:
    min_values_matrix[year] = {}
    avg_values_matrix[year] = {}
    max_values_matrix[year] = {}

    for penetration_level in PENETRATION_LEVELS:
        current_matrix = vsm_matrix[year][penetration_level]
        
        min_value = min(min(current_matrix))
        avg_value = avg(matrix_to_list(current_matrix))
        max_value = max(max(current_matrix))

        if min_value < 0:
            min_value = 0

        if avg_value < 0:
            avg_value = 0

        if max_value < 0:
            max_value = 0

        min_values_matrix[year][penetration_level] = min_value
        avg_values_matrix[year][penetration_level] = avg_value 
        max_values_matrix[year][penetration_level] = max_value

first_penetration_level = PENETRATION_LEVELS[0]

first_penetration_level_data = [(year, penetration_level, avg_value) 
                                for year, penetration_levels_dict in avg_values_matrix.items()
                                for penetration_level, avg_value in penetration_levels_dict.items()
                                if penetration_level == first_penetration_level]
                                

first_level_df = pd.DataFrame(first_penetration_level_data, columns=['Years', 'Penetration Levels', 'Avg Values'])

fig = px.imshow(first_level_df.pivot(index= 'Penetration Levels',
                                     columns= 'Years',
                                     values= 'Avg Values'),
                x= YEARS,
                y= [first_penetration_level],
                labels= dict(color= 'Valores <br> Médios'),
                title= None,
                color_continuous_scale= 'Portland',
                aspect= 'auto')

fig.update_layout(xaxis_title= 'Anos',
                  yaxis_title= 'Quantidade de <br> Eletropostos')

fig.update_layout(yaxis=dict(tickvals= [0],
                             ticktext= [first_penetration_level]))
fig.update_layout(xaxis=dict(tickvals= YEARS,
                             ticktext= YEARS))

for year in YEARS:
    min_value = min_values_matrix[year][first_penetration_level]
    max_value = max_values_matrix[year][first_penetration_level]
    fig.add_annotation(x= year,
                       y= 0,
                       text= '{:.2f}'.format(max_value),
                       showarrow= False,
                       font= dict(color='white'),
                       align='center')

fig.update_layout(height= 250,
                  width= 900)
fig.update_layout(title_x= 0.5)

fig.show()

In [23]:
min_values_matrix = {}
avg_values_matrix = {}
max_values_matrix = {}

for year in YEARS:
    min_values_matrix[year] = {}
    avg_values_matrix[year] = {}
    max_values_matrix[year] = {}

    for penetration_level in PENETRATION_LEVELS:
        current_matrix = vsm_138_matrix[year][penetration_level]
        
        min_value = min(min(current_matrix))
        avg_value = avg(matrix_to_list(current_matrix))
        max_value = max(max(current_matrix))

        if min_value < 0:
            min_value = 0

        if avg_value < 0:
            avg_value = 0

        if max_value < 0:
            max_value = 0

        min_values_matrix[year][penetration_level] = min_value
        avg_values_matrix[year][penetration_level] = avg_value 
        max_values_matrix[year][penetration_level] = max_value

first_penetration_level = PENETRATION_LEVELS[0]

first_penetration_level_data = [(year, penetration_level, avg_value) 
                                for year, penetration_levels_dict in avg_values_matrix.items()
                                for penetration_level, avg_value in penetration_levels_dict.items()
                                if penetration_level == first_penetration_level]
                                

first_level_df = pd.DataFrame(first_penetration_level_data, columns=['Years', 'Penetration Levels', 'Avg Values'])

fig = px.imshow(first_level_df.pivot(index= 'Penetration Levels',
                                     columns= 'Years',
                                     values= 'Avg Values'),
                x= YEARS,
                y= [first_penetration_level],
                labels= dict(color= 'Valores <br> Médios'),
                title= None,
                color_continuous_scale= 'Portland',
                aspect= 'auto')

fig.update_layout(xaxis_title= 'Anos',
                  yaxis_title= 'Quantidade de <br> Eletropostos')

fig.update_layout(yaxis=dict(tickvals= [0],
                             ticktext= [first_penetration_level]))
fig.update_layout(xaxis=dict(tickvals= YEARS,
                             ticktext= YEARS))

for year in YEARS:
    min_value = min_values_matrix[year][first_penetration_level]
    max_value = max_values_matrix[year][first_penetration_level]
    fig.add_annotation(x= year,
                       y= 0,
                       text= '{:.2f}'.format(max_value),
                       showarrow= False,
                       font= dict(color='white'),
                       align='center')

fig.update_layout(height= 250,
                  width= 900)
fig.update_layout(title_x= 0.5)

fig.show()

In [24]:
min_values_matrix = {}
avg_values_matrix = {}
max_values_matrix = {}

for year in YEARS:
    min_values_matrix[year] = {}
    avg_values_matrix[year] = {}
    max_values_matrix[year] = {}

    for penetration_level in PENETRATION_LEVELS:
        current_matrix = vsm_69_matrix[year][penetration_level]
        
        min_value = min(min(current_matrix))
        avg_value = avg(matrix_to_list(current_matrix))
        max_value = max(max(current_matrix))

        if min_value < 0:
            min_value = 0

        if avg_value < 0:
            avg_value = 0

        if max_value < 0:
            max_value = 0

        min_values_matrix[year][penetration_level] = min_value
        avg_values_matrix[year][penetration_level] = avg_value 
        max_values_matrix[year][penetration_level] = max_value

first_penetration_level = PENETRATION_LEVELS[0]

first_penetration_level_data = [(year, penetration_level, avg_value) 
                                for year, penetration_levels_dict in avg_values_matrix.items()
                                for penetration_level, avg_value in penetration_levels_dict.items()
                                if penetration_level == first_penetration_level]
                                

first_level_df = pd.DataFrame(first_penetration_level_data, columns=['Years', 'Penetration Levels', 'Avg Values'])

fig = px.imshow(first_level_df.pivot(index= 'Penetration Levels',
                                     columns= 'Years',
                                     values= 'Avg Values'),
                x= YEARS,
                y= [first_penetration_level],
                labels= dict(color= 'Valores <br> Médios'),
                title= None,
                color_continuous_scale= 'Portland',
                aspect= 'auto')

fig.update_layout(xaxis_title= 'Anos',
                  yaxis_title= 'Quantidade de <br> Eletropostos')

fig.update_layout(yaxis=dict(tickvals= [0],
                             ticktext= [first_penetration_level]))
fig.update_layout(xaxis=dict(tickvals= YEARS,
                             ticktext= YEARS))

for year in YEARS:
    min_value = min_values_matrix[year][first_penetration_level]
    max_value = max_values_matrix[year][first_penetration_level]
    fig.add_annotation(x= year,
                       y= 0,
                       text= '{:.2f}'.format(max_value),
                       showarrow= False,
                       font= dict(color='white'),
                       align='center')

fig.update_layout(height= 250,
                  width= 900)
fig.update_layout(title_x= 0.5)

fig.show()

In [26]:
files_dict = {year: {penetration_level: [] for penetration_level in PENETRATION_LEVELS} for year in YEARS}

min_values_dict = {year: {penetration_level: 100 for penetration_level in PENETRATION_LEVELS} for year in YEARS}
avg_values_dict = {year: {penetration_level: 0 for penetration_level in PENETRATION_LEVELS} for year in YEARS}
max_values_dict = {year: {penetration_level: 0 for penetration_level in PENETRATION_LEVELS} for year in YEARS}

for filename in os.listdir(target_directory := 'Operative_VSMs_by_bar_138'):
    if filename.endswith('.csv'):
        _, _, year_from_file, _, _, penetration_level = filename.split('_')

        year_from_file = int(year_from_file)
        penetration_level = int(penetration_level.split('.')[0])

        files_dict[year_from_file][penetration_level].append(os.path.join(target_directory, filename))

for year in YEARS:
    for penetration_level in PENETRATION_LEVELS:

        avg_values_list = []

        for file in files_dict[year][penetration_level]:
            data = pd.read_csv(file, header= None)

            min_value = max(0, data.min().min())
            avg_value = data.mean().mean()
            max_value = data.max().max()
            
            avg_values_list.append(avg_value) 

            if min_value < min_values_dict[year][penetration_level]:
                min_values_dict[year][penetration_level] = min_value

            avg_values_dict[year][penetration_level] = avg(avg_values_list)

            if max_value > max_values_dict[year][penetration_level]:
                max_values_dict[year][penetration_level] = max_value

avg_values_df = pd.DataFrame(avg_values_dict).head(1)

fig = px.imshow(avg_values_df,
                labels= dict(x= 'Anos',
                            y= 'Quantidade de <br> Eletropostos',
                            color= 'Valores <br> Médios'),
                x=avg_values_df.columns,
                y=avg_values_df.index[::-1],
                color_continuous_scale='Portland',
                aspect= 'auto')

fig.update_layout(yaxis=dict(tickvals= [0],
                             ticktext= [first_penetration_level]))
fig.update_layout(xaxis=dict(tickvals= YEARS,
                             ticktext= YEARS))

for year in YEARS:
    min_annotation = min_values_dict[year][0]
    max_annotation = max_values_dict[year][0]
    fig.add_annotation(x= year,
                       y= 0,
                       text= '{:.2f}<br>{:.2f}'.format(max_annotation, min_annotation),
                       showarrow= False,
                       font= dict(color='white'),
                       align='center')
        
fig.update_layout(height= 250,
                  width= 900)

fig.update_layout(title_x= 0.5)


In [27]:
files_dict = {year: {penetration_level: [] for penetration_level in PENETRATION_LEVELS} for year in YEARS}

min_values_dict = {year: {penetration_level: 100 for penetration_level in PENETRATION_LEVELS} for year in YEARS}
avg_values_dict = {year: {penetration_level: 0 for penetration_level in PENETRATION_LEVELS} for year in YEARS}
max_values_dict = {year: {penetration_level: 0 for penetration_level in PENETRATION_LEVELS} for year in YEARS}

for filename in os.listdir(target_directory := 'Operative_VSMs_by_bar_069'):
    if filename.endswith('.csv'):
        _, _, year_from_file, _, _, penetration_level = filename.split('_')

        year_from_file = int(year_from_file)
        penetration_level = int(penetration_level.split('.')[0])

        files_dict[year_from_file][penetration_level].append(os.path.join(target_directory, filename))

for year in YEARS:
    for penetration_level in PENETRATION_LEVELS:

        avg_values_list = []

        for file in files_dict[year][penetration_level]:
            data = pd.read_csv(file, header= None)

            min_value = max(0, data.min().min())
            avg_value = data.mean().mean()
            max_value = data.max().max()
            
            avg_values_list.append(avg_value) 

            if min_value < min_values_dict[year][penetration_level]:
                min_values_dict[year][penetration_level] = min_value

            avg_values_dict[year][penetration_level] = avg(avg_values_list)

            if max_value > max_values_dict[year][penetration_level]:
                max_values_dict[year][penetration_level] = max_value

avg_values_df = pd.DataFrame(avg_values_dict).head(1)

fig = px.imshow(avg_values_df,
                labels= dict(x= 'Anos',
                            y= 'Quantidade de <br> Eletropostos',
                            color= 'Valores <br> Médios'),
                x=avg_values_df.columns,
                y=avg_values_df.index[::-1],
                color_continuous_scale='Portland',
                aspect= 'auto')

fig.update_layout(yaxis=dict(tickvals= [0],
                             ticktext= [first_penetration_level]))
fig.update_layout(xaxis=dict(tickvals= YEARS,
                             ticktext= YEARS))

for year in YEARS:
    min_annotation = min_values_dict[year][0]
    max_annotation = max_values_dict[year][0]
    fig.add_annotation(x= year,
                       y= 0,
                       text= '{:.2f}<br>{:.2f}'.format(max_annotation, min_annotation),
                       showarrow= False,
                       font= dict(color='white'),
                       align='center')
        
fig.update_layout(height= 250,
                  width= 900)

fig.update_layout(title_x= 0.5)
